In [ ]:

import sqlite3
import pandas as pd
import numpy as np
import datetime as dt
from sqlalchemy import create_engine, text
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import os

RNG = np.random.default_rng(42)


In [ ]:

def generate_coastal_data(start_date="2024-01-01", days=365, freq="1H"):
    """Generate synthetic coastal dataset for multiple sensors."""
    idx = pd.date_range(start=start_date, periods=int(days*24), freq=freq)
    n = len(idx)
    
   
    def tide_series(mean_level=0.5, amp=0.5, daily_amp=0.2):
       
        hours = np.arange(n)
        semidiurnal = amp * np.sin(2 * np.pi * hours / 12.42)  
        daily = daily_amp * np.sin(2 * np.pi * hours / 24.0)
        seasonal = 0.1 * np.sin(2 * np.pi * hours / (24*30))  
        noise = RNG.normal(scale=0.05, size=n)
        return mean_level + semidiurnal + daily + seasonal + noise

    tide1 = tide_series(mean_level=0.6, amp=0.45)
    tide2 = tide_series(mean_level=0.4, amp=0.35) + 0.01*np.linspace(0,1,n)  
   
    temp = 25 + 3*np.sin(2*np.pi*np.arange(n)/(24*180)) + RNG.normal(0,0.8,n)  
    wind_speed = np.abs(5 + 2*np.sin(2*np.pi*np.arange(n)/24) + RNG.normal(0,1,n))
    pressure = 1013 + 5*np.cos(2*np.pi*np.arange(n)/(24*7)) + RNG.normal(0,1.5,n)
    rainfall = RNG.poisson(0.05, n) * (RNG.random(n) < 0.02) * RNG.exponential(2, n)  

    
    turbidity1 = 1 + 0.2*np.sin(np.arange(n)/72) + RNG.normal(0,0.1,n)
    turbidity2 = 1 + 0.25*np.sin(np.arange(n)/60) + RNG.normal(0,0.12,n)
    nitrates = 0.5 + 0.05*np.sin(np.arange(n)/100) + RNG.normal(0,0.02,n)
    
    
    chl = 0.5 + 0.2*np.sin(np.arange(n)/200) + RNG.normal(0,0.05,n)
    
    bloom_times = []
    for day_start in [50, 140, 260]:  
        start_idx = int(day_start*24)
        bloom_len = int(RNG.integers(48, 240))
        end_idx = min(n, start_idx + bloom_len)
        chl[start_idx:end_idx] += np.linspace(0.3, 1.2, end_idx-start_idx) + RNG.normal(0,0.2,end_idx-start_idx)
        bloom_times.append((idx[start_idx], idx[end_idx-1]))

    
    dumping = np.zeros(n, dtype=int)
    dumping_times = []
    for dt_day in [90, 210, 330]:
        i = int(dt_day*24 + RNG.integers(0,12))
        if i < n:
            dumping[i:i+6] = 1  
            turbidity1[i:i+6] += 1.2 + RNG.normal(0,0.2,6)
            dumping_times.append(idx[i])
    
    
    surge = np.zeros(n)
    storm_times = []
    for storm_day in [30, 180, 320]:
        center = int(storm_day*24 + RNG.integers(-6,6))
        width = int(RNG.integers(12,48))
        window = np.exp(-0.5*((np.arange(n)-center)/(width/2))**2)
        amp = RNG.uniform(0.6,1.5)
        surge += amp * window
        wind_speed += 10*window
        pressure -= 20*window
        storm_times.append(idx[center])

    tide1 += surge
    tide2 += surge * 0.9

    df = pd.DataFrame({
        "timestamp": idx,
        "tide_gauge_1_m": tide1,
        "tide_gauge_2_m": tide2,
        "temp_c": temp,
        "wind_m_s": wind_speed,
        "pressure_hpa": pressure,
        "rain_mm": rainfall,
        "turbidity_1": turbidity1,
        "turbidity_2": turbidity2,
        "nitrates_mgL": nitrates,
        "chl_index": chl,
        "illegal_dumping_flag": dumping
    })
    df.set_index("timestamp", inplace=True)

    events = []
    for t in bloom_times:
        events.append({"start": t[0], "end": t[1], "type":"algal_bloom"})
    for t in dumping_times:
        events.append({"time": t, "type":"illegal_dumping"})
    for t in storm_times:
        events.append({"time": t, "type":"storm_surge"})
    return df, pd.DataFrame(events)

df, events_df = generate_coastal_data(start_date="2024-01-01", days=365)
print("Rows:", len(df))
df.head()


     





Rows: 8760


C:\Users\kinara\AppData\Local\Temp\ipykernel_1748\1775868780.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  idx = pd.date_range(start=start_date, periods=int(days*24), freq=freq)


,tide_gauge_1_m,tide_gauge_2_m,temp_c,wind_m_s,pressure_hpa,rain_mm,turbidity_1,turbidity_2,nitrates_mgL,chl_index,illegal_dumping_flag
timestamp,,,,,,,,,,,
2024-01-01 00:00:00,0.615236,0.470904,25.526621,5.121843,1017.327924,0.0,1.038735,1.136376,0.518840,0.382075,0
2024-01-01 01:00:00,0.818702,0.624521,24.919510,5.519375,1016.866998,0.0,0.974367,0.970565,0.530812,0.528125,0
2024-01-01 02:00:00,1.120769,0.853031,25.331545,6.107397,1017.000378,0.0,0.968410,1.311520,0.489593,0.551096,0
2024-01-01 03:00:00,1.240433,0.859286,25.419095,7.444114,1016.978917,0.0,1.045385,1.048196,0.512899,0.516315,0
2024-01-01 04:00:00,1.083800,0.961766,25.295467,5.761465,1016.253104,0.0,0.891145,0.802587,0.508001,0.531624,0


In [ ]:

DB_PATH = "coastal_ew_system.db"
engine = create_engine(f"sqlite:///{DB_PATH}", echo=False)

df.reset_index().to_sql("sensor_readings", engine, if_exists="replace", index=False)
events_df.to_sql("historical_events", engine, if_exists="replace", index=False)


def ingest_batch(batch_df):
    """Append a DataFrame batch to sensor_readings."""
    batch_df.reset_index().to_sql("sensor_readings", engine, if_exists="append", index=False)


with engine.connect() as conn:
    count = conn.execute(text("SELECT COUNT(*) FROM sensor_readings")).scalar()
print("Total records in sensor_readings:", count)



Total records in sensor_readings: 8760


In [ ]:

rdf = pd.read_sql("SELECT * FROM sensor_readings", engine, parse_dates=["timestamp"], index_col="timestamp")

rdf["tide1_roll_6h"] = rdf["tide_gauge_1_m"].rolling(6).mean()
rdf["tide1_roll_24h"] = rdf["tide_gauge_1_m"].rolling(24).mean()
rdf["chl_24h_mean"] = rdf["chl_index"].rolling(24).mean()
rdf["turbidity_6h_max"] = rdf[["turbidity_1","turbidity_2"]].max(axis=1).rolling(6).max()
rdf["wind_6h_max"] = rdf["wind_m_s"].rolling(6).max()
rdf["sea_level_trend_7d"] = rdf["tide_gauge_1_m"].rolling(24*7).apply(lambda x: np.polyfit(np.arange(len(x)), x, 1)[0] if len(x)>2 else 0)
rdf = rdf.fillna(method="bfill").fillna(method="ffill")


rdf["rule_high_tide_alert"] = ((rdf["tide_gauge_1_m"] - rdf["tide1_roll_24h"]) > 0.8) | (rdf["tide_gauge_1_m"] > 1.4)
rdf["rule_bloom_alert"] = (rdf["chl_index"] > (rdf["chl_24h_mean"] + 0.4)) & (rdf["chl_index"] > 0.9)
rdf["rule_dumping_alert"] = rdf["illegal_dumping_flag"] == 1
rdf["rule_storm_alert"] = (rdf["wind_6h_max"] > 18) & (rdf["pressure_hpa"] < 995)

rdf[["tide_gauge_1_m","tide1_roll_24h","rule_high_tide_alert","chl_index","chl_24h_mean","rule_bloom_alert"]].tail()



C:\Users\kinara\AppData\Local\Temp\ipykernel_1748\219511300.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rdf = rdf.fillna(method="bfill").fillna(method="ffill")


,tide_gauge_1_m,tide1_roll_24h,rule_high_tide_alert,chl_index,chl_24h_mean,rule_bloom_alert
timestamp,,,,,,
2024-12-30 19:00:00,0.229468,0.670179,False,0.510594,0.450595,False
2024-12-30 20:00:00,0.491606,0.662440,False,0.501105,0.454338,False
2024-12-30 21:00:00,0.802457,0.659596,False,0.413030,0.454026,False
2024-12-30 22:00:00,0.937364,0.656864,False,0.450174,0.454867,False
2024-12-30 23:00:00,1.047476,0.658612,False,0.493650,0.454950,False


In [ ]:

features = ["tide_gauge_1_m","tide_gauge_2_m","wind_m_s","pressure_hpa","rain_mm","turbidity_1","turbidity_2","nitrates_mgL","chl_index"]
X = rdf[features].copy()


scaler = StandardScaler()
Xs = scaler.fit_transform(X)

split_idx = int(len(Xs)*0.6)
clf = IsolationForest(n_estimators=200, contamination=0.01, random_state=42)
clf.fit(Xs[:split_idx])

scores = clf.decision_function(Xs)  
anomaly_flag = clf.predict(Xs)  
rdf["anomaly_score"] = -scores  
rdf["anomaly_flag_iforest"] = (anomaly_flag == -1).astype(int)

print("Anomalies detected:", rdf["anomaly_flag_iforest"].sum())
rdf["anomaly_flag_iforest"].value_counts()


Anomalies detected: 80


anomaly_flag_iforest
0    8680
1      80
Name: count, dtype: int64

In [ ]:

rdf["composite_alert"] = (
    rdf["rule_high_tide_alert"] |
    rdf["rule_bloom_alert"] |
    rdf["rule_dumping_alert"] |
    rdf["rule_storm_alert"] |
    (rdf["anomaly_flag_iforest"]==1)
)

def alert_severity(row):
    score = 0
    score += 2 if row["rule_storm_alert"] else 0
    score += 2 if row["rule_dumping_alert"] else 0
    score += 1 if row["rule_bloom_alert"] else 0
    score += 1 if row["rule_high_tide_alert"] else 0
    score += 1 if row["anomaly_flag_iforest"] else 0
    return min(score,4)

rdf["alert_severity"] = rdf.apply(alert_severity, axis=1)
rdf[["composite_alert","alert_severity","anomaly_flag_iforest","rule_bloom_alert","rule_dumping_alert","rule_storm_alert"]].tail()



,composite_alert,alert_severity,anomaly_flag_iforest,rule_bloom_alert,rule_dumping_alert,rule_storm_alert
timestamp,,,,,,
2024-12-30 19:00:00,False,0,0,False,False,False
2024-12-30 20:00:00,False,0,0,False,False,False
2024-12-30 21:00:00,False,0,0,False,False,False
2024-12-30 22:00:00,False,0,0,False,False,False
2024-12-30 23:00:00,False,0,0,False,False,False


In [ ]:

import pytz
tz = pytz.timezone("UTC")


def get_alerts(start, end):
    s = rdf.loc[start:end]
    a = s[s["composite_alert"]==True]
    return a.sort_index()


start_picker = widgets.DatePicker(description='Start', value=rdf.index.min().to_pydatetime().date())
end_picker = widgets.DatePicker(description='End', value=rdf.index.min().to_pydatetime().date() + dt.timedelta(days=7))
refresh_btn = widgets.Button(description="Refresh", button_style='info')
alerts_out = widgets.Output()
plot_out = widgets.Output()


def send_alerts_simulated(alert_rows, channels=["sms","app","web"]):
    """Simulates sending alerts. Replace body with Twilio/Firebase API calls in production."""
    
    results = []
    for ts, row in alert_rows.iterrows():
        msg = {
            "timestamp": str(ts),
            "severity": int(row["alert_severity"]),
            "reason": ",".join([k for k in ["rule_bloom_alert","rule_dumping_alert","rule_storm_alert","rule_high_tide_alert","anomaly_flag_iforest"] if row.get(k)]),
            "channels": channels
        }
     
        results.append(msg)
    return results

def on_refresh(b):
    with plot_out:
        clear_output(wait=True)
        s = start_picker.value
        e = end_picker.value
        if s is None or e is None:
            print("Pick start and end dates.")
            return
        s_dt = pd.to_datetime(s).tz_localize(None)
        e_dt = pd.to_datetime(e) + pd.Timedelta(days=1) - pd.Timedelta(seconds=1)
        subset = rdf.loc[s_dt:e_dt]
        if subset.empty:
            print("No data in that range.")
            return
       
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=subset.index, y=subset["tide_gauge_1_m"], name="Tide 1 (m)"))
        fig.add_trace(go.Scatter(x=subset.index, y=subset["chl_index"], name="Chl index", yaxis="y2", opacity=0.8))
       
        alerts = subset[subset["composite_alert"]==True]
        fig.add_trace(go.Scatter(x=alerts.index, y=alerts["tide_gauge_1_m"], mode='markers', name='Alerts', marker=dict(size=8, symbol="x", color="red")))
        
        fig.update_layout(
            yaxis=dict(title='Tide (m)'),
            yaxis2=dict(title='Chl index', overlaying='y', side='right'),
            height=450,
            title=f"Coastal sensors {s_dt.date()} to {e_dt.date()}"
        )
        fig.show()
    with alerts_out:
        clear_output(wait=True)
        a = get_alerts(s_dt, e_dt)
        if a.empty:
            print("No alerts in selected range.")
        else:
            display(a[["tide_gauge_1_m","chl_index","alert_severity","anomaly_flag_iforest","rule_bloom_alert","rule_dumping_alert","rule_storm_alert"]].head(50))
            
            send_btn = widgets.Button(description=f"Send {len(a)} Alerts (simulate)", button_style='danger')
            def on_send(b2):
                results = send_alerts_simulated(a)
                print("Simulated send (no external calls). Sample payloads:")
                for i, r in enumerate(results[:5]):
                    print(json.dumps(r))
            send_btn.on_click(on_send)
            display(send_btn)

refresh_btn.on_click(on_refresh)
ui = widgets.VBox([widgets.HBox([start_picker, end_picker, refresh_btn]), plot_out, alerts_out])
display(ui)


on_refresh(None)
